# Introducing Rendervous  

Rendervous is a project built on Vulkan and PyTorch that models processes—potentially graphics-related—as differentiable maps whenever possible. These maps define parametric, vector-valued functions as follows:  

$$  
f_\theta: \mathcal{R}^n \rightarrow \mathcal{R}^m \quad \text{where} \quad y = f_\theta(x)  
$$  

Each map includes a forward method to compute $y$ and a backward mechanism to propagate gradients, $\partial L / \partial y$, back to both $x$ and $\theta$.

Unlike conventional tensor-based differentiation frameworks, Rendervous focuses on operator-level abstraction rather than an operation-result graph. Depending on the map's specifics, gradient computation may either be automated or involve replaying the forward evaluation. This design requires careful consideration when combining maps to balance execution complexity effectively.

Let's start importing necessary libraries.

In [1]:
try: # install all dependencies in colab 
    import google.colab
    !sudo apt-get update -y
    !sudo apt-get install -y libnvidia-gl-555 vulkan-tools glslang-tools vulkan-validationlayers-dev
    !pip install pyav==13.1.0
    !pip install git+https://github.com/rendervous/rendervous_project.git
except:
    print("Executing locally")

Executing locally


In [2]:
import rendervous as rdv
import torch

We will start with the simplest map in Rendervous: a constant map. This map represents functions in the form:  

$$  
f_\theta(x) := \theta  
$$  

For example, the constant map $f(x)=(1, 2)$ could be expressed as: 

In [3]:
my_consmap = rdv.const[1.0, 2.0]

Some maps can be defined in a generic form. For instance, the map `my_consmap` has a fixed output dimension of 2, but its input dimension can vary. The method `cast` allows you to set specific input and output dimensions, returning a new map that is linked to the same parameters or sub-maps.

In [4]:
my_non_generic_consmap = my_consmap.cast(input_dim=4)
## This would trigger an assertion error
# print(my_non_generic_consmap(torch.rand(10, 2, device=rdv.device())))

In [5]:
print(my_non_generic_consmap(torch.rand(10, 4, device=rdv.device())))

tensor([[1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


If the map is generic at the input it can be evaluated and will take the input dimension from the last dimension of the argument.

In [6]:
print(my_consmap(torch.rand(10, 2, device=rdv.device())))  # no problem here.

tensor([[1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


Some special constants (non-trainable) can be instantiated directly. 

In [7]:
zero = rdv.ZERO
one = rdv.ONE

Another interested behaviour in rendervous is the promotion of scalar-valued functions (i.e., functions with output dimension 1).
Those functions behave as if they were generic in the output since they can be cast to any output dimension.

In [8]:
print(zero.cast(output_dim=3)(torch.rand(5, 4)))

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


In case a map with generic output is evaluated, a single value output is assumed. 

In [9]:
print(one(torch.rand(5,4)))

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)


## Operations  

Unlike tensor-based operations and automatic differentiation, in Rendervous, operators applied to maps create new maps. For example, the following creates a map representing the addition of two maps. Specifically, it defines a map that extracts the direction $\omega$ and the position $x$ from a ray input in the form $x|\omega$. The resulting map computes the ray's end-point at a distance of $1$ (assuming the direction is normalized).  


In [10]:
my_map = rdv.ray_direction + rdv.ray_position

In [11]:
rays = torch.randn(10, 6)
print(rays)
ray_ends = my_map(rays)
print(ray_ends)

tensor([[ 0.0729, -1.0733,  0.2339, -0.1594, -0.1786,  0.5086],
        [ 0.1383, -0.0694, -0.1429,  0.9510,  0.0054,  0.1359],
        [-0.3524, -0.0915,  0.5617,  0.0774,  0.7373, -1.5320],
        [-1.5755,  0.5641,  0.1582,  1.3421, -1.3772, -0.2621],
        [-0.2502,  1.8547, -1.2233, -1.8539, -0.8393, -1.4448],
        [-1.0413,  1.1743,  1.1825,  0.8703,  0.5924,  0.1326],
        [-0.1620, -0.6319,  0.7656, -0.3028,  0.4848,  0.0693],
        [-0.6915, -0.9721, -0.8586,  1.1571, -1.1840,  0.6471],
        [-1.1810,  0.3683,  0.9229, -0.5546, -1.6430, -1.0633],
        [-1.2968, -0.2598, -0.8825, -0.9643,  1.4711,  0.4838]])
tensor([[-0.0864, -1.2519,  0.7426],
        [ 1.0893, -0.0640, -0.0070],
        [-0.2750,  0.6458, -0.9703],
        [-0.2335, -0.8131, -0.1040],
        [-2.1041,  1.0154, -2.6681],
        [-0.1710,  1.7666,  1.3151],
        [-0.4648, -0.1470,  0.8349],
        [ 0.4655, -2.1561, -0.2116],
        [-1.7356, -1.2747, -0.1405],
        [-2.2611,  1.2113,

In [12]:
rays[:,0:3] + rays[:,3:6]

tensor([[-0.0864, -1.2519,  0.7426],
        [ 1.0893, -0.0640, -0.0070],
        [-0.2750,  0.6458, -0.9703],
        [-0.2335, -0.8131, -0.1040],
        [-2.1041,  1.0154, -2.6681],
        [-0.1710,  1.7666,  1.3151],
        [-0.4648, -0.1470,  0.8349],
        [ 0.4655, -2.1561, -0.2116],
        [-1.7356, -1.2747, -0.1405],
        [-2.2611,  1.2113, -0.3986]])

Also, when operating a map with a scalar value it is converted to a constant map automatically. Notice the use of `|` as concat operator.

In [13]:
opposed_ray = rdv.ray_position | (-1 * rdv.ray_direction)
print(opposed_ray(rays))

tensor([[ 0.0729, -1.0733,  0.2339,  0.1594,  0.1786, -0.5086],
        [ 0.1383, -0.0694, -0.1429, -0.9510, -0.0054, -0.1359],
        [-0.3524, -0.0915,  0.5617, -0.0774, -0.7373,  1.5320],
        [-1.5755,  0.5641,  0.1582, -1.3421,  1.3772,  0.2621],
        [-0.2502,  1.8547, -1.2233,  1.8539,  0.8393,  1.4448],
        [-1.0413,  1.1743,  1.1825, -0.8703, -0.5924, -0.1326],
        [-0.1620, -0.6319,  0.7656,  0.3028, -0.4848, -0.0693],
        [-0.6915, -0.9721, -0.8586, -1.1571,  1.1840, -0.6471],
        [-1.1810,  0.3683,  0.9229,  0.5546,  1.6430,  1.0633],
        [-1.2968, -0.2598, -0.8825,  0.9643, -1.4711, -0.4838]],
       device='cuda:0', grad_fn=<AutogradMapFunctionBackward>)
